### Entity Component System

In [1]:
from dataclasses import dataclass
import dataclasses
import esper
import time
import sys

### Component

Code is from esper headless example.

`__slots__` are used to reduce the object size and save memory.

In [2]:
class Velocity:
    __slots__ = ['x', 'y', 'foo', 'bar']
    def __init__(self, x=0, y=0, foo=0, bar=0):
        self.x = x
        self.y = y
        self.foo = 0
        self.bar = 0

        
class Position:
    __slots__ = ['x', 'y']
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

have to define `__repr__` manually:

In [7]:
Velocity()

instead of writing __init__() and their attributes all the time, we can use a dataclass! Its a new feature in python 3.7. dataclass is backported to 3.6 if you want to use it in advance. `pip install dataclass`.

In [12]:
def add_slots(cls):
    if '__slots__' in cls.__dict__:
        raise TypeError(f'{cls.__name__} already specifies __slots__')

    cls_dict = dict(cls.__dict__)
    field_names = tuple(f.name for f in dataclasses.fields(cls))
    cls_dict['__slots__'] = field_names
    for field_name in field_names:
        cls_dict.pop(field_name, None)
    cls_dict.pop('__dict__', None)
    qualname = getattr(cls, '__qualname__', None)
    cls = type(cls)(cls.__name__, cls.__bases__, cls_dict)
    if qualname is not None:
        cls.__qualname__ = qualname
    return cls

In [11]:
@add_slots
@dataclass
class Velocity:
    x : float = 0.0
    y : float = 0.0
    foo: float = 0.0
    bar: float = 0.0


@add_slots        
@dataclass
class Position:
    x : float = 0.0
    y : float = 0.0

`__repr__` is defined automatically.

In [9]:
Velocity()

Velocity(x=0.0, y=0.0, foo=0.0, bar=0.0)

Currently slots are not offically supported, but we can add them by use this funky decorator!
(https://github.com/ericvsmith/dataclasses/blob/master/dataclass_tools.py). In practice, I will add the slots at the end of the project. "Premature Optimization Is the Root of All Evil".

### Processor

In [ ]:
class MovementProcessor(esper.Processor):
    def __init__(self):
        super().__init__()

    def process(self):
        for ent, (vel, pos) in self.world.get_components(Velocity, Position):
            pos.x += vel.x
            pos.y += vel.y
            print("Current Position: {}".format((int(pos.x), int(pos.y))))

In [16]:
world = esper.World()
world.get_components(Velocity, Position)

<generator object World.get_components at 0x048D0300>

### World

In [ ]:
def main():
    # Create a World instance to hold everything:
    world = esper.World()

    # Instantiate a Processor (or more), and add them to the world:
    movement_processor = MovementProcessor()
    world.add_processor(movement_processor)

    # Create entities, and assign Component instances to them:
    player = world.create_entity()
    world.add_component(player, Velocity(x=0.9, y=1.2))
    world.add_component(player, Position(x=5, y=5))

    # A dummy main loop:
    try:
        while True:
            # Call world.process() to run all Processors.
            world.process()
            time.sleep(1)

    except KeyboardInterrupt:
        return

In [ ]:
main()